In [1]:
import pandas as pd
import json
import os # Operaciones relacionadas con el sistema de archivos
import datetime

### Metadatos

In [2]:
# Función para leer y filtrar los archivos JSON
def filtrar_archivos_json(directorio, categoria_busqueda):
    ''' Esta función, llamada filtrar_archivos_json, se encarga de buscar y filtrar 
        archivos JSON en un directorio específico en función de una categoría de búsqueda 
        proporcionada.   '''
    resultados = []
    for archivo in os.listdir(directorio):
            ruta_archivo = os.path.join(directorio, archivo)

            # Leer el contenido del archivo JSON
            with open(ruta_archivo) as f:
                archivo_json = f.readlines()

            # Cargar archivo JSON linea por linea
            for linea in archivo_json:
                data = json.loads(linea)
                # Verificar si la categoría buscada está presente en el archivo JSON
                categorias = data.get("category", [])
                if isinstance(categorias, list) and any(categoria_busqueda.lower() in categoria.lower() for categoria in categorias):
                    resultados.append(data)
    return resultados


In [87]:
# Filtrar los archivos JSON para la categoría "restaurant"
restaurant = filtrar_archivos_json('../PROYECTO_FINAL_EQUIPO/Google/metadata-sitios', "restaurant")
# Convertir los resultados a un pandas DataFrame
df_restaurant = pd.DataFrame(restaurant)

In [88]:
df_restaurant.shape

(121637, 15)

In [89]:
df_restaurant.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Fresh Off The Truck,"Fresh Off The Truck, Mueller Trailer Eatery, 4...",0x8644b5f995211beb:0x489b703481320404,"Food truck located at Mueller Trailer Eats, se...",30.296518,-97.707304,"[Caterer, Restaurant]",5.0,8,None,None,{'Service options': ['Delivery']},None,"[0x8644b5f3d971856f:0x3e04f8904bf2b2ee, 0x8644...",https://www.google.com/maps/place//data=!4m2!3...
1,Papa Joe's BBQ Co.,"Papa Joe's BBQ Co., 24415 NE 10th Ave, Ridgefi...",0x5495aef37bce66d5:0xfcf97974639cd5dd,None,45.798254,-122.660751,[Restaurant],4.6,14,None,"[[Thursday, 12–7PM], [Friday, 12–7PM], [Saturd...",{'From the business': ['Identifies as women-le...,Permanently closed,"[0x5495b2cea4d3a7f7:0xd788b88074f97be6, 0x5495...",https://www.google.com/maps/place//data=!4m2!3...
2,Eldric's Restaurant,"Eldric's Restaurant, 401 W International Airpo...",0x56c89792537f15c7:0xff259d38a6f51288,None,61.173896,-149.891410,[Filipino restaurant],4.5,98,$,"[[Thursday, 11AM–7PM], [Friday, 11AM–7PM], [Sa...","{'Service options': ['Curbside pickup', 'Takeo...",Open ⋅ Closes 7PM,"[0x56c897aa943ce3bd:0x6ba11eda584e11bf, 0x56c8...",https://www.google.com/maps/place//data=!4m2!3...


In [90]:
#1061 restaurantes duplicados
df_restaurant['gmap_id'].duplicated().sum()

1061

In [91]:
#Revisión de restaurantes duplicados
restaurantes_duplicados = df_restaurant[df_restaurant['gmap_id'].duplicated(keep=False)]
# Ordena los restaurantes duplicados por gmap_id
restaurantes_duplicados_ordenados = restaurantes_duplicados.sort_values(by='gmap_id')
restaurantes_duplicados_ordenados.head(2)


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
36445,Bar 91,"Bar 91, 400 Ave of the Champions, Palm Beach G...",0x146c6711a70d7991:0xe478335f760c8ccc,None,26.828991,-80.141201,"[Bar & grill, Bar, Restaurant, Sports bar]",3.9,8,None,"[[Wednesday, 6:30AM–9PM], [Thursday, 6:30AM–9P...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 6:30AM Thu,"[0x88ded52e4b87e21d:0x773ff6613ec82a36, 0x88de...",https://www.google.com/maps/place//data=!4m2!3...
37506,Bar 91,"Bar 91, 400 Ave of the Champions, Palm Beach G...",0x146c6711a70d7991:0xe478335f760c8ccc,None,26.828991,-80.141201,"[Bar & grill, Bar, Restaurant, Sports bar]",3.9,8,None,"[[Wednesday, 6:30AM–9PM], [Thursday, 6:30AM–9P...","{'Service options': ['Outdoor seating', 'Takeo...",Closed ⋅ Opens 6:30AM Thu,"[0x88ded52e4b87e21d:0x773ff6613ec82a36, 0x88de...",https://www.google.com/maps/place//data=!4m2!3...


In [92]:
#Eliminar duplicados
df_restaurant = df_restaurant.drop_duplicates(subset='gmap_id') 

In [93]:
#Tenemos 120,575 restaurantes con 15 atributos, de los cuales es necesario cuales tienen
#realmente reseñas para los futuros procesos de ML
df_restaurant.shape

(120576, 15)

### Reviews-estados

Se hace la carga de las Reviews para obtener las que pertenezcan a los restaurantes
y así mismo filtrar los restaurantes con Review. 

In [11]:
ruta_reviews_estados = "../PROYECTO_FINAL_EQUIPO/Google/reviews-estados/"
rutas_subfolders = [f.path for f in os.scandir(ruta_reviews_estados) if f.is_dir()]

In [12]:
def df_json(ruta_folder):
    '''Unión de archivos en un DataFrame '''
# Inicializa una lista para almacenar los objetos JSON de los archivos
    data = []
    
    # Itera a través de los archivos en la carpeta especificada
    for file_name in os.listdir(ruta_folder):
        # Construye la ruta completa de cada archivo
        file_path = os.path.join(ruta_folder, file_name)
        # Abre el archivo en modo lectura ('r')
        with open(file_path, 'r') as file:
            # Itera a través de las líneas del archivo
            for line in file:
                # Elimina espacios en blanco al principio y al final de la línea
                line = line.strip()
                # Intenta cargar la línea como un objeto JSON
                json_obj = json.loads(line)
                    # Agrega el objeto JSON a la lista de datos
                data.append(json_obj)

    # Crea un DataFrame de pandas a partir de la lista de datos
    df = pd.DataFrame(data)
    
    # Devuelve el DataFrame resultante
    return df

In [13]:
def df_json_general(subfolders):
    # Inicializa una lista para almacenar DataFrames individuales
    df_list = []
    
    # Itera a través de las subcarpetas especificadas
    for subfolder in subfolders:
        # Llama a la función read_json_files para procesar los archivos JSON en la subcarpeta
        df = df_json(subfolder)
        
        # Agrega el DataFrame resultante a la lista
        df_list.append(df)
    
    # Devuelve una lista de DataFrames, uno por cada subcarpeta
    return df_list

In [122]:

lista_df = df_json_general(rutas_subfolders)

In [123]:
df_reviews = pd.concat(lista_df, ignore_index=True)

In [124]:
df_reviews.shape

(8879925, 8)

In [125]:
df_reviews.sample(3)

,user_id,name,time,rating,text,pics,resp,gmap_id
6834552,117644209318792895049,Damian Montaño,1484520412090,5,None,None,None,0x89babc3ca5db1f21:0x29a8dbf0efec5c09
7299069,116736933972425145669,R Matt Branson,1613581748058,3,None,None,None,0x89b2d74f85c287ff:0x40e46d4d45d6bcf1
4969178,118432990171013081993,Darren Ursua,1554937978043,5,Fusion tacos but great stuff.,None,None,0x80c8c7f42091bb83:0xc023765e6e20deab


#### SELECCIÓN DE RESTAURANTES CON RESEÑAS

In [126]:
#Obtenemos los establecimientos que tienen Reviews
id_establecimientos_con_reviews = df_reviews["gmap_id"].unique()

In [127]:
#Se tienen 142,234 establecimientos únicos o que tienen una review
id_establecimientos_con_reviews.shape[0]

142234

In [128]:
df_restaurant.shape

(120576, 15)

In [129]:
#Filtrar los restaurantes que tienen Reviews.
df_restaurantes = df_restaurant[df_restaurant["gmap_id"].isin(id_establecimientos_con_reviews)].reset_index(drop=True)

In [158]:
df_restaurantes.shape

(3223, 15)

In [162]:
df_restaurantes.to_csv('Restaurantes_Google.csv',index=False)

#### Filtrar reseñas de Restaurantes

In [138]:
filtro=df_restaurantes['gmap_id'].unique()

In [140]:
df_reviews_res = df_reviews[df_reviews["gmap_id"].isin(filtro)]

In [143]:
df_reviews_res.shape

(152797, 8)

In [163]:
df_reviews_res.to_csv('Restaurantes_Google_Reviews.csv',index=False)

Revision

In [147]:
df_reviews_res.sample(3)

,user_id,name,time,rating,text,pics,resp,gmap_id
2965873,105099992938275555206,R P,1562330140327,4,None,None,None,0x89e252dad6359927:0xe7c6ebeb64fba4a1
5379603,108473292045516936214,Or B.,1468092477040,5,None,None,None,0x80c8c0abbf3c0181:0xd5d4d30179b63405
2210438,104536943205250498382,Donna Wendler,1545620160217,1,Typical Mc D food.,None,None,0x88067b7dc59f6271:0x65b8c8bd7ce4f6da


In [152]:
df_restaurantes[df_restaurantes['gmap_id']=='0x88067b7dc59f6271:0x65b8c8bd7ce4f6da']

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
704,McDonald's,"McDonald's, 111 E Tyranena Park Rd, Lake Mills...",0x88067b7dc59f6271:0x65b8c8bd7ce4f6da,"Classic, long-running fast-food chain known fo...",43.090169,-88.908613,"[Fast food restaurant, Breakfast restaurant, C...",3.5,598,$,"[[Sunday, 4:30AM–11PM], [Monday, 4:30AM–11PM],...","{'Service options': ['Curbside pickup', 'Drive...",Open ⋅ Closes 11PM,"[0x88067b7e4fdc361d:0xd4ab8f3c5eb8666a, 0x8806...",https://www.google.com/maps/place//data=!4m2!3...


In [153]:
df_reviews_res[df_reviews_res['gmap_id']=='0x88067b7dc59f6271:0x65b8c8bd7ce4f6da'].shape

(458, 8)

In [154]:
df_reviews_res.sample(2)

,user_id,name,time,rating,text,pics,resp,gmap_id
7019651,111863426724828952386,Christina Tyree,1620952598570,5,None,None,None,0x89b13d433b8fe3ab:0xd2ca0857a62cbf06
1408184,113049193067944699398,David Stehman,1581205514561,5,Good food enjoy,None,None,0x883a722a5b4b8227:0x948406a7f9f4c469


''' '''


In [165]:
df_reviews_res.shape

(152797, 8)